In [1]:
import os

import imageio
import pandas as pd
from keras_preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os

import imageio
import pandas as pd
from keras import Input, Model
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.optimizers import Adam
import numpy as np
import keras.utils
from sklearn.metrics import accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from keras.layers import (Conv2D, Flatten, Lambda, Dense, concatenate,
                          Dropout, Input)
from keras.models import Model
import numpy as np
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, Flatten, Dense
from keras.optimizers import Adam


data_image = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data\\train_images"
data_dir = "C:\\Users\\Priit\\Dropbox\\Informaatika\\Magister\\Tehisnärvivõrgud\\data"

label_column = "AdoptionSpeed"

Using TensorFlow backend.


In [2]:
train = pd.read_csv(os.path.join(data_dir, "train.csv"), sep=',')
pet_ids = train["PetID"]

selected_columns = ["Type",
                    "Gender",
                    "Color1",
                    "Color2",
                    "Color3",
                    "MaturitySize",
                    "FurLength",
                    "Vaccinated",
                    "Dewormed",
                    "Sterilized",
                    "Health",
                    "State",
                    "MaturitySize"]




y = train[label_column]

# One-Hot-encode
X = pd.get_dummies(train[selected_columns], columns=selected_columns)

# Normalize:
to_normalize = ["Age", "Fee", "Quantity"]
for to_norm in to_normalize:
    X[to_norm] = (train[to_norm] - train[to_norm].mean()) / train[to_norm].std()

X = train.assign(paths=[os.path.join(data_dir, data_image, pet_id + "-1.jpg") for pet_id in pet_ids])

# Convert labels to str
X[label_column] = X[label_column].astype(str)

X_test = X.iloc[:400, :]
X_train = X.iloc[400:, :]
X_train = X_train.reset_index()
X_test = X_test.reset_index()
#X["paths"]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


In [3]:
data_gen_args = dict(featurewise_center=True,
                     featurewise_std_normalization=True,
                     rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     zoom_range=0.2)
# TODO: fill
data_gen_args = dict()

height, width = 320, 320
batch = 8

train_imggen = ImageDataGenerator(data_gen_args) #rescale=1./255, rotation_range=30, horizontal_flip=True)

train_loader = train_imggen.flow_from_dataframe(X_train, x_col='paths',
                                                y_col=label_column,                                         
                                                target_size=(height, width), 
                                                class_mode='categorical', 
                                                validation_split=0.2,
                                                batch_size=batch)

test_imggen = ImageDataGenerator(data_gen_args)
test_loader = train_imggen.flow_from_dataframe(X_test, 
                                               x_col='paths',
                                               y_col=label_column,                                         
                                               target_size=(height, width), 
                                               class_mode='categorical', 
                                               batch_size=batch)

#train_imggen.fit()
#test_imggen.fit()

Found 14267 images belonging to 5 classes.
Found 385 images belonging to 5 classes.


In [4]:
inputs = Input(shape=(height, width, 3))
h = Conv2D(32, (7, 7), strides=(2, 2))(inputs)
h = Activation('relu')(h)
h = Flatten()(h)
h = Dense(5)(h)
predictions = Activation('softmax')(h)

model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_loader, epochs=1, steps_per_epoch=10)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
10/10 [==============================] - ETA: 12s - loss: 12.0890 - acc: 0.250 - ETA: 7s - loss: 12.0888 - acc: 0.250 - ETA: 5s - loss: 12.7603 - acc: 0.20 - ETA: 3s - loss: 12.0887 - acc: 0.25 - ETA: 3s - loss: 12.4916 - acc: 0.22 - ETA: 2s - loss: 12.0887 - acc: 0.25 - ETA: 1s - loss: 12.0886 - acc: 0.25 - ETA: 1s - loss: 12.0886 - acc: 0.25 - ETA: 0s - loss: 11.8648 - acc: 0.26 - 5s 518ms/step - loss: 11.8871 - acc: 0.2625


In [5]:
#pred = model.predict_generator(test_loader, steps=len(X_test.index), verbose=1)
predictions = []
y_true = []

#for x, y in tqdm(test_loader):
    #y_true.append(np.argmax(y))
    #predictions.append(np.argmax(model.predict(x)))

batch_index = 0
while batch_index <= test_loader.batch_index:
    data = test_loader.next()
    #data_list.append(data[0])
    batch_index = batch_index + 1
    print(batch_index)
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [ ]:
print("Kappa on test: {}".format(round(cohen_kappa_score(y_true, predictions, weights="quadratic"), 4)))
print("Accuracy on test: {}".format(round(accuracy_score(y_true, predictions), 4)))

In [ ]:
# https://stackoverflow.com/questions/49618986/neural-network-in-keras-with-two-different-input-types-images-and-values
# TODO: read images in
# TODO: read features in
image = np.random.rand(10, 66, 200, 3)
feature = np.random.rand(10, 3)  # feature vector
y = np.random.normal(0, 1, (10, 1))

image_input = Input(shape=(66, 200, 3))
aux_input = Input(shape=(3,))

lamb = Lambda(lambda x: x / 127.5 - 1.0, input_shape=(66, 200, 3))(image_input)
cov1 = Conv2D(24, 5, 5, activation='elu', subsample=(2, 2))(lamb)
cov2 = Conv2D(36, 5, 5, activation='elu', subsample=(2, 2))(cov1)
cov3 = Conv2D(48, 5, 5, activation='elu', subsample=(2, 2))(cov2)
cov4 = Conv2D(64, 3, 3, activation='elu')(cov3)
cov5 = Conv2D(64, 3, 3, activation='elu')(cov4)
dropout = Dropout(0.5)(cov5)
flatten = Flatten()(dropout)

# Here we add in the feature vectors
merge = concatenate([flatten, aux_input])

d1 = Dense(100, activation='elu')(merge)
d2 = Dense(50, activation='elu')(d1)
d3 = Dense(10, activation='elu')(d2)
out = Dense(1)(d3)

model = Model(inputs=[image_input, aux_input], outputs=[out])
model.compile(loss='mse', optimizer='rmsprop')
model.fit([image, feature], y)


In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('first_try.h5')